# Pyquil Tutorials

**Warning:** The pyquil binder is deprecated for python 3.6, trying to execute this notebook without a recent enough version of python will only yield exceptions.

We have made a binder that converts QLM circuits into pyquil circuits, we have also made the `PyquilQPU` class which is a qpu wrapper around pyquil's simulator/real chip which takes QLM jobs, converts them to pyquil jobs, executes them on a pyquil simulator/real quantum chip, then returns a QLM Result.

## QLM to Pyquil conversion

A function that converts a QLM circuit into a pyquil circuit

In [ ]:
from qat.interop.pyquil.converters import qlm_to_pyquil
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
pyquil_circuit = qlm_to_pyquil(qlm_circuit)

print(pyquil_circuit)

## Pyquil to QLM conversion

A function that converts a Pyquil circuit into a QLM circuit

In [ ]:
from qat.interop.pyquil.converters import pyquil_to_qlm
from pyquil import Program
from pyquil.gates import H, CNOT, MEASURE
from qat.core.util import get_syntax

nbqubits = 2

prog = Program()

creg = prog.declare("ro", "BIT", 2)
prog += H(0)
prog += CNOT(0, 1)
prog += MEASURE(0, creg[0])
prog += MEASURE(1, creg[1])

# get result with included measures
qlm_circuit = pyquil_to_qlm(prog)

for index, op in enumerate(qlm_circuit.ops):
    print("Gate {} with params {} on qubits {} and cbits {}".format(*get_syntax(qlm_circuit, index), op.cbits))

# get result with separated measures
print("---------------------------------------------------------")
qlm_circuit, to_measure = pyquil_to_qlm(prog, sep_measures=True)

for index, op in enumerate(qlm_circuit.ops):
    print("Gate {} with params {} on qubits {}".format(*get_syntax(qlm_circuit, index)))
print("qubits to measure {}".format(to_measure))

## PyquilQPU

The wrapper around whatever pyquil simulator or quantum chip you wish to use.
In this example we use the [qvm](https://github.com/rigetti/qvm) that we have running as server. We used this same method to show the qiskit connector working with pyquilqpu to execute qiskit code on pyquil's qvm and retrieving the result in QLM format.<br>
**Warning:** This will only yield an error if there is no qvm running.

In [ ]:
import os
from qat.interop.pyquil.providers import PyquilQPU
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

from pyquil import get_qc

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

# get a QuantumComputer object from the vqm
os.environ["QCS_SETTINGS_APPLICATIONS_PYQUIL_QVM_URL"] = "http://127.0.0.1:15011"
os.environ["QCS_SETTINGS_APPLICATIONS_PYQUIL_QUILC_URL"] = "tcp://127.0.0.1:15012"
qvm = get_qc('9q-qvm')

# create the QPU from the QuantumComputer object
qpu = PyquilQPU(qvm)

result = qpu.submit(qlm_job)
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))
